In [27]:
# This code is an adaptation of https://stackoverflow.com/questions/3637350/how-to-write-stereo-wav-files-in-python
# I wrote the first half of it, as indicated in the comments
# Most of the mathematics come from the original, most of the notes handling is mine

# This code produces a wav file containing the melody for "Fra' Martino", Brother John
# in English or Frère Jacques in French. 

import numpy as np
import wave
import struct
###################
# IMPORTANT: Comment the next line if not using jupyter
from IPython.display import Audio 
###################

OCTAVE_FACTOR = 1   # Modify this parameter to change octave.
                    # Every increment by a factor of 2 shifts the notes by one octave
                    # Example: set to 2 instead of 1 to shift to the higher octave,
                    # or set to 4 instead of 1 to shift to the second higher octave
                    # Negative exponents to 2^n are also accepted
                
# Table containing frequencies for all notes in an octave, in Italian notation
# The octave containing the tuning note "LA", A = 440 Hz, is taken as a reference
DO =    261.6*OCTAVE_FACTOR
RE =    293.7*OCTAVE_FACTOR
MI =    329.6*OCTAVE_FACTOR
FA =    349.2*OCTAVE_FACTOR
SOL =   392.0*OCTAVE_FACTOR
LA =    440.0*OCTAVE_FACTOR
SI =    493.9*OCTAVE_FACTOR
DO_UP = 523.3*OCTAVE_FACTOR

four_notes = 32000 # How long does a "battuta", bar, lasts

DOREMI = np.zeros(four_notes)
DOREMI[:four_notes//4] = DO
DOREMI[four_notes//4:four_notes//2] = RE
DOREMI[four_notes//2:3*four_notes//4] = MI
DOREMI[3*four_notes//4:-10] = DO # up to -10 so that it stops immediately before starting the next bar

MIFASOL = np.zeros(four_notes)
MIFASOL[:four_notes//4] = MI
MIFASOL[four_notes//4:four_notes//2] = FA
MIFASOL[four_notes//2:3*four_notes//4] = SOL
MIFASOL[3*four_notes//4:-10] = SOL

SOLLASOLFAMIDO = np.zeros(four_notes)
SOLLASOLFAMIDO[:four_notes//8] = SOL
SOLLASOLFAMIDO[four_notes//8:four_notes//4] = LA
SOLLASOLFAMIDO[four_notes//4:3*four_notes//8] = SOL
SOLLASOLFAMIDO[3*four_notes//8:four_notes//2] = FA
SOLLASOLFAMIDO[four_notes//2:3*four_notes//4] = MI
SOLLASOLFAMIDO[3*four_notes//4:-10] = DO

RESOLDO = np.zeros(four_notes)
RESOLDO[:four_notes//4] = RE
RESOLDO[four_notes//4:four_notes//2] = SOL
RESOLDO[four_notes//2:3*four_notes//4] = DO
RESOLDO[3*four_notes//4:-10] = DO

RESOLDO_UP = np.zeros(four_notes)
RESOLDO_UP[:four_notes//4] = RE
RESOLDO_UP[four_notes//4:four_notes//2] = SOL
RESOLDO_UP[four_notes//2:3*four_notes//4] = DO_UP
RESOLDO_UP[3*four_notes//4:-10] = DO_UP

freq = np.append(DOREMI,DOREMI)
freq = np.append(freq,MIFASOL)
freq = np.append(freq,MIFASOL)
freq = np.append(freq,SOLLASOLFAMIDO)
freq = np.append(freq,SOLLASOLFAMIDO)
freq = np.append(freq,RESOLDO)
freq = np.append(freq,RESOLDO_UP)

data_size = freq.shape[0] # How many samples do we have, computed on the basis of the melody that was coded
fname = "fra_martino.wav"

# DISCLAIMER:
# The mathematics contained below come from the original code
# I do not understand enough maths to write the Fourier transform myself
# ENDOFDISCLAIMER

frate = 11025.0  # framerate as a float
amp = 64000.0     # multiplier for amplitude

sine_list_x = []
for x in range(data_size):
    sine_list_x.append(np.sin(2*np.pi*freq[x]*(x/frate)))

print(max(sine_list_x),min(sine_list_x))

with wave.open(fname, "w") as wav_file:

    nchannels = 1
    sampwidth = 2
    framerate = int(frate)
    nframes = data_size
    comptype = "NONE"
    compname = "not compressed"

    wav_file.setparams((nchannels, sampwidth, framerate, nframes,
        comptype, compname))

    for s in sine_list_x:
        # write the audio frames to file
        wav_file.writeframes(struct.pack('h', int(s*amp/2)))

print('Done')

######################
# IMPORTANT: Comment the next line if not using jupyter
Audio(fname) 
######################

0.9999999995940128 -0.9999999963461136
Done
